In [1]:
from langchain.llms import GooglePalm

In [9]:
import os
import streamlit as st
import pickle
import time
import langchain
from langchain import OpenAI
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
import requests
from bs4 import BeautifulSoup

In [21]:
import pandas as pd

In [6]:
with open("google_key.txt", "r") as f:
    api_key=f.read()

In [7]:
llm=GooglePalm(google_api_key=api_key, temperature=0.7)

In [12]:
urls=["https://www.livemint.com/news/india/uttarkashi-tunnel-collapse-silkyara-ndrf-deploys-team-rescue-mission-41-workers-top-10-updates-pushkar-singh-dhami-11700703183514.html",
     "https://timesofindia.indiatimes.com/city/kochi/centre-calls-health-meet-as-covid-rises-in-kerala-10-deaths-this-month/articleshow/106106851.cms"]
responses=[]

for url in urls:
    response=requests.get(url)
    if response.status_code==200:
        soup = BeautifulSoup(response.text, 'html.parser')
        text = soup.get_text()
        responses.append((text, response.url))
        
    else:
        print(f"Failed to retrieve the URL. Url={url} Status_code :{response.status_code}")

In [13]:
len(responses)

2

In [14]:
import string 
punct=string.punctuation
seps=[]
for i in punct:
    seps.append(i)

In [16]:
text_splitter=RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", "?","." ]+seps+[" "],
    chunk_size=1000,
    chunk_overlap=200
)

In [17]:
splitted_tuple=[]
for i in responses:
    splitted_text= text_splitter.split_text(i[0])
    for j in splitted_text:
        splitted_tuple.append((j, i[1]))

In [18]:
data_dict=[{"chunks": content, "url": url} for content, url in splitted_tuple]

In [22]:
df = pd.DataFrame(data_dict)

In [75]:
def filter_chunks(text):
    text=text.lower().strip()
    text=text.split(" ")
    words=[]
    for i in text:
        if i not in seps:
            word="".join([j for j in i if j not in seps])
            words.append(word)
    sentence=" ".join(words)
    return sentence  

In [76]:
filter_chunks("add! .")

'add'

In [80]:
df["chunks"]=df["chunks"].apply(filter_chunks)

In [82]:
df.dropna()

,chunks,url
0,uttarkashi tunnel collapse ndrf team enters tu...,https://www.livemint.com/news/india/uttarkashi...
1,explore sign in epaper subscribe tuesday 19...,https://www.livemint.com/news/india/uttarkashi...
2,top sections news india news world ...,https://www.livemint.com/news/india/uttarkashi...
3,a opinion markets stock markets c...,https://www.livemint.com/news/india/uttarkashi...
4,multimedia collections videos webstories...,https://www.livemint.com/news/india/uttarkashi...
5,,https://www.livemint.com/news/india/uttarkashi...
6,gainers losers tue dec 19 2023 123054 top gai...,https://www.livemint.com/news/india/uttarkashi...
7,business news news ind...,https://www.livemint.com/news/india/uttarkashi...
8,a 15member ndrf team led by a commandant has b...,https://www.livemint.com/news/india/uttarkashi...
9,2 medical professionals have been stationed wi...,https://www.livemint.com/news/india/uttarkashi...


In [ ]:
df["chunks"]=df["chunks"].apply()

In [54]:
csv_path='dataset.csv'
if os.path.exists(csv_path):
    os.remove(csv_path)

In [55]:
df.to_csv(csv_path, index=False)

In [67]:
csv_data=pd.read_csv("dataset.csv", encoding="cp437")

In [68]:
csv_data

,chunks,url
0,Uttarkashi Tunnel Collapse: NDRF team enters t...,https://www.livemint.com/news/india/uttarkashi...
1,"Explore Sign in e-paper Subscribe Tuesday, ...",https://www.livemint.com/news/india/uttarkashi...
2,+ Top Sections News India News Worl...,https://www.livemint.com/news/india/uttarkashi...
3,&A Opinion Markets Stock Markets ...,https://www.livemint.com/news/india/uttarkashi...
4,+ MultiMedia Collections Videos WebStori...,https://www.livemint.com/news/india/uttarkashi...
5,.,https://www.livemint.com/news/india/uttarkashi...
6,Gainers & Losers Tue Dec 19 2023 12:30:54 Top...,https://www.livemint.com/news/india/uttarkashi...
7,Business News/ News / ...,https://www.livemint.com/news/india/uttarkashi...
8,"A 15-member NDRF team, led by a commandant, ha...",https://www.livemint.com/news/india/uttarkashi...
9,2. Medical professionals have been stationed w...,https://www.livemint.com/news/india/uttarkashi...


In [69]:
from langchain.document_loaders.csv_loader import CSVLoader

In [70]:
loader=CSVLoader(file_path=csv_path, source_column='chunks')

In [71]:
data=loader.load()

UnicodeDecodeError: 'charmap' codec can't decode byte 0x8f in position 3392: character maps to <undefined>